In [1]:
import requests,configparser
def get_cookie():
    config = configparser.ConfigParser()
    config.read('secrets.txt')
    cookie = config['session_info']['cookie']
    return cookie
def get_inputs(day):
    cookie, day = get_cookie(), int(day)
    headers = {'session': cookie}
    url = f'https://adventofcode.com/2021/day/{day}/input'
    session = requests.Session()
    resp = session.get(url,cookies=headers)
    return resp.text.split('\n')[:-1]

In [ ]:
# DAY1
def count_increases(data,depth):
    increases = 0
    for i in range(depth,len(data)):
        increases += 1 if data[i]>data[i-depth] else 0
    return increases

data_day1 = [int(x) for x in get_inputs(1) if x!='']
print(f'Part 1: {count_increases(data_day1,1)}')
print(f'Part 2: {count_increases(data_day1,3)}')

In [ ]:
# DAY2
class Submarine():
    def __init__(self,h_pos,depth,has_aim=False,aim=0):
        self.h_pos = h_pos
        self.depth = depth
        self.has_aim = has_aim
        self.aim = aim
    def forward(self,x):
        self.h_pos += int(x)
        self.depth += int(x)*self.aim
    def down(self,x):
        if self.has_aim:
            self.aim += int(x)
        else:
            self.depth += int(x)
    def up(self,x):
        if self.has_aim:
            self.aim -= int(x)
        else:
            self.depth -= int(x)
    def take_command(self,command):
        command_map = {'up': self.up, 'down':self.down, 'forward':self.forward}
        directions = command.split(' ')    # e.g. direction = ['forward','2']
        move_method = command_map.get(directions[0])
        move_method(directions[1])
    def get_position(self):
        return (self.h_pos,self.depth)
    def navigate(self,data):
        for command in data:
            self.take_command(command)
        return self
    def get_pos_multip(self):
        return self.h_pos*self.depth

data_day2 = get_inputs(2)

d2p1 = Submarine(0,0)
print(f'Part 1: {d2p1.navigate(data_day2).get_pos_multip()}')

d2p2 = Submarine(0,0,True,0)
print(f'Part 2: {d2p2.navigate(data_day2).get_pos_multip()}')


In [ ]:
# Day 3
from statistics import mode
data_day3 = get_inputs(3)

def to_matrix(data):
    counters = []
    for l in data:
        counters.append(list(l))
    return counters

def bin_diag(data, idx, most = 1, reduce=False):
    L = len(data[0])
    if len(data) == 1:
        return sum([int(x)*(2**i) for i,x in enumerate(data[0][:idx-L-1:-1])])
    if idx == len(data[0]):
        return 0
    
    ones = [e[idx] for e in data].count('1')
    if ones >= len(data)/2:
        winner = '1' if most else '0'
    else:
        winner = '0' if most else '1'    
    if reduce:
        data = list(filter(lambda x: (x[idx] == winner), data))
    
    return int(winner)*(2**(L-idx-1))+bin_diag(data, idx+1, most, reduce)

def to_decimal(l):
    return sum([int(x)*(2**i) for i,x in enumerate(l[::-1])])

# Part 1
gamma = bin_diag(to_matrix(data_day3), 0, most = 1, reduce=False)
epsilon = bin_diag(to_matrix(data_day3), 0, most = 0, reduce=False)
print(f'Part 1: {gamma*epsilon}')

# Part 2
oxg = bin_diag(to_matrix(data_day3), 0, most = 1, reduce=True)
co2 = bin_diag(to_matrix(data_day3), 0, most = 0, reduce=True)
print(f'Part 2: {oxg*co2}')

In [ ]:
# Day 4
data_day4 = get_inputs(4)
import re

def solve_day4(data):
    B = Bingo(data)
    # Part 1
    unmarked_totals, last_number = B.find_winnig_grid()
    print(f'Part 1: {unmarked_totals*last_number}')
    
    # Part 2
    unmarked_totals, last_number = B.find_last_winning_grid()
    print(f'Part 2: {unmarked_totals*last_number}')

def get_sequence(data):
    sequence = [int(i) for i in data[0].split(',')]
    return sequence

def get_grids(data): 
    grids = []
    N = len(re.findall(r'[\d]+',data[2]))
    row= 2
    while row < len(data):
        g = [[int(i) for i in re.split('[ ]+',data[j].strip())] for j in range(row,row+5)]
        row += N+1
        grids.append(g)
    return grids

class Bingo():
    def __init__(self,data):
        self.sequence = get_sequence(data)
        self.grids = [Grid(g,self) for g in get_grids(data)]
    
    def find_winnig_grid(self):
        # Find the grid that requires the smallest `winner_idx` in the sequence
        L = min([(idx,grid.find_winner_idx()) for idx,grid in enumerate(self.grids)],key=lambda x:x[1])
        return self.calculate_unmarked_total(L[0],L[1]),self.sequence[L[1]]
    
    def find_last_winning_grid(self):
        # Find the grid that requires the largest `winner_idx` in the sequence
        L = max([(idx,grid.find_winner_idx()) for idx,grid in enumerate(self.grids)],key=lambda x:x[1])
        return self.calculate_unmarked_total(L[0],L[1]),self.sequence[L[1]]
    
    def calculate_unmarked_total(self,grid_idx,seq_idx):
        return self.grids[grid_idx].sum_unmarked(seq_idx)

class Grid():
    def __init__(self,data,B):
        self.grid = data
        self.size = len(data)
        self.B = B
    
    def get_rows(self):    return [(*i,) for i in self.grid]
    def get_row(self,i):   return self.get_rows(i)
    def get_cols(self):    return list(zip(*self.grid))
    def get_col(self,i):   return self.get_cols(i)
    
    def find_winner_idx(self):
        idx = 999
        rows_and_cols = list(set().union(self.get_rows(), self.get_cols()))
        for r in rows_and_cols:
            w = check_for_win(r,self.B.sequence)
            idx = w if (0<w<idx) else idx
        return idx
    
    def sum_unmarked(self,idx):
        s = 0
        for row in self.get_rows():
            s += sum([x for x in row if x not in self.B.sequence[:idx+1]])
        return s
        
def check_for_win(l,sequence):
    matches = [idx for idx, element in enumerate(sequence) if element in l]
    if len(matches)==len(l):
        return max(matches)
    return -1

solve_day4(data_day4)

In [ ]:
# Day 5
import operator,collections,re
from functools import reduce

def transform_day5(data):
    to_list = [list(map(lambda x:int(x), list(re.findall('(\d+)',i)))) for i in data]
    return list(map(lambda x: list(zip(x[::2],x[1::2])),to_list))

def test_data_day5():
    d5 = '''0,9 -> 5,9
        8,0 -> 0,8
        9,4 -> 3,4
        2,2 -> 2,1
        7,0 -> 7,4
        6,4 -> 2,0
        0,9 -> 2,9
        3,4 -> 1,4
        0,0 -> 8,8
        5,5 -> 8,2
        '''.split('\n')[:-1]
    return d5

class Line():
    def __init__(self,points):
        self.start , self.end = points
    def is_H_or_V(self):
        return any(self.start[i] == self.end[i] for i in range(2))
    def get_points(self):
        points = []
        direction = tuple(0 if self.end[i]==self.start[i] else int(math.copysign(1,self.end[i]-self.start[i])) for i in range(2))
        distance = 1+ max([abs(self.end[i]-self.start[i]) for i in range(2)])
        for d in range(distance):
            new_point = tuple(map(lambda i, j: i + d*j, self.start, direction))
            points.append(new_point)
        return points
        
def solve_day5(data):
    lines = [Line(i) for i in transform_day5(data)]
    
    # Part 1 - horizontal or vertical lines
    all_hv_points = []
    for line in lines:
        if line.is_H_or_V():
            all_hv_points += line.get_points()
    # calculate the frequency of all points
    points_freq = collections.Counter(all_hv_points)
    # print out number of points with frequency more than 1
    print(f'Part 1: {len(list(filter(lambda x: x[1]>1,points_freq.items())))}')
    
    # Part 2 - all lines
    all_points = []
    for line in lines:
        all_points += line.get_points()
    all_points_freq = collections.Counter(all_points)
    print(f'Part 2: {len(list(filter(lambda x: x[1]>1,all_points_freq.items())))}')

solve_day5(get_inputs(5))

In [7]:
# Day 6
import re, collections
test_d6 = ['3,4,3,1,2']

class School():
    def __init__(self,data):
        fish_list = [int(x) for x in re.findall('\d+',data[0])]
        self.fish_dict = self.to_dict(fish_list)
        self.day = 0
    def to_dict(self,f):
        d = collections.Counter(f)
        for i in range(9):
            d[i] = d[i] or 0
        return d
    def one_day(self):
        self.day += 1
        tomorrow_dict = dict(map(lambda x: (x,0),list(range(9))))
        for k,v in self.fish_dict.items():
            if k>=1:
                tomorrow_dict[k-1] = v
        tomorrow_dict[8] += self.fish_dict[0]
        tomorrow_dict[6] += self.fish_dict[0]
        return tomorrow_dict
    def age(self,days):
        for i in range(days):
            self.fish_dict = self.one_day()
        return sum(self.fish_dict.values())
        
s = School(get_inputs(6))
print(f'Part 1: {s.age(80)} lanternfishes on day #{s.day}')
print(f'Part 2: {s.age(256 - 80)} lanternfishes on day #{s.day}')

Part 1: 385391 lanternfishes on day #80
Part 2: 1728611055389 lanternfishes on day #256
